In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
from IPython.display import clear_output

import quick_pp.las_handler as las

In [3]:
from tkinter import Tk, filedialog

root = Tk()
file_objects = filedialog.askopenfiles(title='Choose well Log ASCII Standard (LAS) files to be combined',
                                        filetype=(('LAS Files', '*.LAS *.las'), ('All Files', '*.*')),
                                        multiple=True,
                                        mode='rb')
root.destroy()
if file_objects:
    # Test read_las_file function
    merged_df, _ = las.read_las_files(file_objects)
clear_output()

Only engine='normal' can read wrapped files
c:\projects\quick_pp\venv310\lib\site-packages\welly\curve.py:470: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  if self.df.index.is_numeric() and not self.df.index.empty:
c:\projects\quick_pp\venv310\lib\site-packages\welly\curve.py:930: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(self.df.iloc[:, 0]) or pd.api.types.is_string_dtype(self.df.iloc[:, 0]):
c:\projects\quick_pp\venv310\lib\site-packages\welly\curve.py:930: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(self.df.iloc[:, 0]) or pd.api.types.is_string_dtype(self.df.iloc[:, 0]):
c:\projects\quick_pp\venv310\lib\site-packages\welly\curve.py:930: FutureWarning: is_categ

In [5]:
# TODO: Rename and filter columns

In [ ]:
# TODO:  Export to parquet